# LLM Movie Recommender

Denne notebooken demonstrerer bruk av en språkmodel (Language Model - LM) og en vektordatabase for anbefaling av filmer.

Denne metoden løser det såkalte "cold start" problemet for anbefalingsalgoritmer, hvor det ikke finnes
data fra brukeren av systemet eller data fra andre brukere av systemet til å gi anbefalinger.

I tilfellet hvor det finnes data på hvilke brukere som foretrekker hvilke filmer, er det enklere å lage en anbefalingsalgoritme.
Da er det mulig å bruke denne dataen til å gi anbefalinger - for eksempel, blandt alle brukere som har gitt høy vurdering (rating)
for "Star Wars", er det kanskje gitt høy rating for "Back to The Future", og da kan man gi anbefaling av denne.

I stedet, kan vi bruke en språkmodell til å finne film-anbefalinger basert på hvor likt sammendraget av filmen er (semantisk likhet), og basert på hvor ofte
navnet på filmen forekommer i samme kontekst som andre filmer.

Denne anbefalingsalgoritmen ligner på clustering. Datasettet består av unlabelled data - informasjon om filmer, uten informasjon
om hvilke filmer som er like eller hvilke brukere som like hvilke filmer. Algoritmen gir informasjon om hvor like filmer er hverandre,
og kan dermed brukes til å gi anbefalinger. For eksempel kan vi forvente (håpe) på at filmer som handler å reise i tid ved hjelp av tidsmaskiner
ligger nær hverandre - det vil si at cluster-algoritmen mener disse filmene har en likhet med hverandre.

La oss først begynne med å hente filmdata fra den åpne filmdatabasen TMDb ved å bruke API-et.

Dette er veldig rett fram. Vi henter informasjon om alle tilgjengelige filmer som har 1000 brukervurderinger eller mer. 

In [24]:
%autoreload 2
from main import *
from pprint import pprint

In [4]:
from config import TMDB_API_KEY
import requests

tmdb_url = "https://api.themoviedb.org/3"

In [5]:
??fetch_popular_movies

Signature: fetch_popular_movies(min_votes: int = 1000) -> list[dict]
Source:   
def fetch_popular_movies(min_votes: int = 1_000) -> list[dict]:
    """
    Fetch most popular movies from themoviedb (TMDb)

    Parameters
    ----------
    min_votes
        Only fetch movies that have this number of votes or more

    Returns
    -------
    A list of movies, each movie in a dictionary
    """
    # Set the API endpoint and parameters
    url = tmdb_url + "/discover/movie"

    # Fetch movies page by page until there are no more
    movies = []
    i = 1
    while True:
        params = {
            "api_key": TMDB_API_KEY,
            "sort_by": "vote_average.desc",
            "vote_count.gte": min_votes,
            "page": i,
        }

        # Send a GET request to the API and raise an exception if it fails
        response = requests.get(url, params=params)
        response.raise_for_status()

        # Extract the movie data from the response
        movies_ = response.json()

In [6]:
movies = fetch_popular_movies()

API-et returnerer grunnleggende informasjon om filmene. Av interesse her er tittel, sjanger, år, og sammendrag.


In [25]:
pprint(movies[0])

{'adult': False,
 'backdrop_path': '/zfbjgQE1uSd9wiPTX4VzsLi0rGG.jpg',
 'genre_ids': [18, 80],
 'id': 278,
 'original_language': 'en',
 'original_title': 'The Shawshank Redemption',
 'overview': 'Imprisoned in the 1940s for the double murder of his wife and '
             'her lover, upstanding banker Andy Dufresne begins a new life at '
             'the Shawshank prison, where he puts his accounting skills to '
             'work for an amoral warden. During his long stretch in prison, '
             'Dufresne comes to be admired by the other inmates -- including '
             'an older prisoner named Red -- for his integrity and '
             'unquenchable sense of hope.',
 'popularity': 176.351,
 'poster_path': '/9cqNxx0GxF0bflZmeSMuL5tnGzr.jpg',
 'release_date': '1994-09-23',
 'title': 'The Shawshank Redemption',
 'video': False,
 'vote_average': 8.705,
 'vote_count': 26355}



Her er følgende av interesse:
- Tittel
- Filmsjanger
- Sammendrag
- År

Vi samler denne informasjonen i en dataframe for enklere og raskere prosessering.
Vi bruker `polars` som et raskere og mer moderne alternativ til `pandas` selv om det har lite å bety i denne sammenheng
siden datasettet er så lite.

Først henter vi filmsjangernavn fra API-et og bruker disse istedet for filmsjanger-ID'er.

Deretter samler vi informasjonen vi ønsker å bruke til å sammenligne i en kolonne ("text") - nemlig  tittel, år, sammendrag og sjangere.

In [8]:
import polars as pl
from functools import lru_cache

??get_id_to_genre.__wrapped__

Signature: get_id_to_genre.__wrapped__() -> dict[int, str]
Source:   
@lru_cache
def get_id_to_genre() -> dict[int, str]:
    """Return a mapping from genre id to genre name"""
    # Fetch genre names
    url = tmdb_url + "/genre/movie/list"
    params = {"api_key": TMDB_API_KEY}
    response = requests.get(url, params=params)
    response.raise_for_status()
    id_and_genre = response.json()["genres"]

    # Map genre ids to genre names
    id_to_genre = {x["id"]: x["name"] for x in id_and_genre}

    return id_to_genre
File:      ~/code/llm-movie-recommender/main.py
Type:      function

In [9]:
??prep_movies

Signature: prep_movies(movies: list[dict]) -> polars.dataframe.frame.DataFrame
Source:   
def prep_movies(movies: list[dict]) -> pl.DataFrame:
    """
    Prepare movie data for embedding

    Attributes
    ----------
    movies
        List of movies

    Returns
    -------
    polars.DataFrame
    """
    # Create a Polars DataFrame from the movie data
    df = pl.DataFrame(movies)

    id_to_genre = get_id_to_genre()
    # Add genre names from genre ids
    df_genres = df["genre_ids"].map_elements(
        lambda li: [id_to_genre[id_] for id_ in li], return_dtype=pl.List(pl.String)
    )
    df = df.with_columns(df_genres.alias("genres"))

    # Extract year from release date
    df = df.with_columns(df["release_date"].str.head(4).alias("year"))

    # Merge all information into a single text column
    text = (
        "Movie title: "
        + df["title"]
        + ".\nYear: "
        + df["year"]
        + ".\nOverview: "
        + df["overview"]
        + "\nGenres: "
        

In [10]:
df = prep_movies(movies)

La oss se på innholdet til DataFramen

In [11]:
len(df)

4188

In [12]:
df.head(1)

id,title,year,overview,genres,text
str,str,str,str,list[str],str
"""278""","""The Shawshank …","""1994""","""Imprisoned in …","[""Drama"", ""Crime""]","""Movie title: T…"


Dette gir følgende tekst for filmen. Det er denne teksten vi skal bruke i vektordatabasen til å finne lignende filmer.

In [13]:
print(df[0, "text"])

Movie title: The Shawshank Redemption.
Year: 1994.
Overview: Imprisoned in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.
Genres: Drama, Crime.


Vi instansierer en vektordatabase som vi skal bruke til å finne liknende dokumenter. Her finnes det mange alternativer, og vi har valgt ChromaDB hvor vi kan velge en valgfri embedding-funksjon.

In [14]:
import chromadb
client = chromadb.Client()

Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Lag en collection og legg til dokumenter.
`chromadb` gjør all jobb  automatisk. Først tokeniseres dokumentteksten til tokens, før den blir embeddet av en innebygd språkmodell og deretter indeksert av databasen slik at den effektivt kan finne nærliggende naboer til dokumentet. Embeddingen fanger essensen av syntaktisk og semantisk mening til hele dokumentet.

Det mulig å bruke en custom funksjon for å embedde dokumenter. Som standard brukes det en LM som heter `all-MiniLM-L6-v2` fra librariet Sentence Transformer. Dette er en språkmodell på 23 millioner paremetere trent på 1 milliard tokens (ord / sub-ord) som outputter 384 dimensionale vektorer. Til sammenligning er GPT-4 fra OpenAI 1.7 billioner paremetre og trent på 13 billioner tokens og har 3072 dimensionale setnings-vektorer.


In [15]:
title_collection = client.get_or_create_collection("movie_titles")

In [16]:
if not title_collection.count():
    title_collection.add(documents=df["title"].to_list(), ids=df["id"].to_list())

In [32]:
embedding = title_collection.query(query_texts=["Star Wars"], n_results=1, include=["embeddings"])["embeddings"]
embedding = embedding[0][0]
print(len(embedding))

384


In [33]:
print(embedding[:10])

[-0.0732547789812088, 0.010070987045764923, -0.0035544871352612972, -0.013902664184570312, -0.04536248371005058, 0.010174545459449291, 0.06956266611814499, 0.029881272464990616, 0.0726681649684906, 0.04335178807377815]


Databasen tilbyr flere alternativer for å måle likheten mellom dokumenter. Likheten mellom to dokumenter er gitt av distansen mellom de to tilhørende vektor embeddingene. Som default bruker ChromaDB $L^2$ distanse, gitt av:

$ d=∑_i(A_i - B_i)^2 $

Hvor $A_i$ er index $i$ i vektor embeddingen til dokument $A$

In [17]:
results = title_collection.query(query_texts=["Star Wars"], n_results=20)

In [18]:
print(results["documents"])

[['Star Wars', 'Star Wars: The Last Jedi', 'Star Wars: The Force Awakens', 'Star Wars: The Clone Wars', 'Star Wars: The Rise of Skywalker', 'Star Wars: Episode I - The Phantom Menace', 'Starship Troopers', 'Rogue One: A Star Wars Story', 'Return of the Jedi', 'Empire of the Sun', 'Guardians of the Galaxy', 'Solo: A Star Wars Story', 'Star Trek', 'The Empire Strikes Back', 'Star Wars: Episode III - Revenge of the Sith', 'Interstellar', 'Guardians of the Galaxy Vol. 2', 'King Kong', 'King Kong', 'Star Wars: Episode II - Attack of the Clones']]


Språkmodellen har fanget god semantisk betydning kun utifra filmtittelen. "Star Wars" har høy likhet til "Return of the Jedi" da begge er Star Wars filmer men har svak syntaktisk likhet. Vi ser også at LMen henter ut andre Sci-Fi filmer som "Star Trek" og "Interstellar".

For å demonstrere at LMen gir større likhet mellom dokumenter av lik semantisk betydning, legger vi til dokumentet "Star Alliance". I motsetning til tidligere dokumenter, er ikke dette en film, men navn på et selskap som er en en allianse av flyselskaper.

In [19]:
title_collection.add(documents=["Star Alliance"], ids=["9999"])

In [20]:
results = title_collection.query(query_texts=["Star Wars"], n_results=20)

In [21]:
print(results["documents"])

[['Star Wars', 'Star Wars: The Last Jedi', 'Star Wars: The Force Awakens', 'Star Wars: The Clone Wars', 'Star Wars: The Rise of Skywalker', 'Star Wars: Episode I - The Phantom Menace', 'Starship Troopers', 'Rogue One: A Star Wars Story', 'Return of the Jedi', 'Empire of the Sun', 'Guardians of the Galaxy', 'Solo: A Star Wars Story', 'Star Trek', 'The Empire Strikes Back', 'Star Wars: Episode III - Revenge of the Sith', 'Star Alliance', 'Interstellar', 'Guardians of the Galaxy Vol. 2', 'King Kong', 'King Kong']]


Databasen gir relativt svak likhet mellom "Star Wars" og "Star Alliance" til tross for at de har høy syntaktisk likhet. 

La oss lage en collection som bruker hele filmbeskrivelsen

In [34]:
collection = client.get_or_create_collection("movies")
if not collection.count():
    collection.add(documents=df["text"].to_list(), ids=df["id"].to_list())

In [43]:
results = collection.query(query_texts=["Back to the future"], n_results=20)

In [44]:
for d in results["documents"][0]: print(d, '\n')

Movie title: Back to the Future.
Year: 1985.
Overview: Eighties teenager Marty McFly is accidentally sent back in time to 1955, inadvertently disrupting his parents' first meeting and attracting his mother's romantic interest. Marty must repair the damage to history by rekindling his parents' romance and - with the help of his eccentric inventor friend Doc Brown - return to 1985.
Genres: Adventure, Comedy, Science Fiction. 

Movie title: Back to the Future Part II.
Year: 1989.
Overview: Marty and Doc are at it again in this wacky sequel to the 1985 blockbuster as the time-traveling duo head to 2015 to nip some McFly family woes in the bud. But things go awry thanks to bully Biff Tannen and a pesky sports almanac. In a last-ditch attempt to set things straight, Marty finds himself bound for 1955 and face to face with his teenage parents -- again.
Genres: Adventure, Comedy, Science Fiction. 

Movie title: Back to the Future Part III.
Year: 1990.
Overview: The final installment of the Bac

De aller fleste anbefalingene treffer ganske bra. De fleste dreier seg om reise i tid eller tidsmaskiner, self om det er noen anbefalinger som ikke virker til å være relevant og som antageligvis har blitt inkludert på grunn av øvrig syntaktisk likhet. Dette kan en håpe forbedrer seg med en større språkmodell hvor en større andel av embeddingen fanger semtantisk informasjon kontra syntaktisk informasjon.

forklar kontrastiv læring